In [177]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import torch.nn.init as init

# Create the NN model

In [210]:
class ClassificationModel(nn.Module): 
        
    def __init__(self, nodes_per_layer, input_shape, num_classes):
        super(ClassificationModel, self).__init__()

        layers = []
        # input layer with the input dimension of the data shape        
        layers.append(nn.Linear(input_shape, nodes_per_layer[0]))
        # He initialization for the first layer
        #init.kaiming_uniform_(layers[0].weight, mode='fan_in', nonlinearity='relu')
        # the activation function for each node
        layers.append(nn.ReLU()) 
        # hidden layers with dimension of nodes_per_layer
        for i in range(1, len(nodes_per_layer)):
            layers.append(nn.Linear(nodes_per_layer[i - 1], nodes_per_layer[i]))
            # He initialization for the hidden layers
            #init.kaiming_uniform_(layers[-1].weight, mode='fan_in', nonlinearity='relu')
            layers.append(nn.ReLU())
        # output layer with the binary output
        layers.append(nn.Linear(nodes_per_layer[-1], num_classes))
        layers.append(nn.Softmax(dim=1))

        self.model = nn.Sequential(*layers)
                
  
    def forward(self, x):
        x = x.to(self.model[0].weight.dtype)
        return self.model(x)
    
    def train_model(self, train_loader, num_epochs=5, lr=0.001, val_loader=True) :
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=lr)

        for epoch in range(num_epochs):
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = self(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            
            if val_loader is not None:
                self.eval()
                correct = 0
                total = 0
                with torch.no_grad():
                    for val_inputs, val_labels in val_loader:
                        val_outputs = self(val_inputs)
                        _, predicted = torch.max(val_outputs.data, 1)
                        total += val_labels.size(0)
                        correct += (predicted == val_labels).sum().item()

                accuracy = correct / total
            
            print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}, Validation Accuracy: {accuracy}')
            
    def predict(self, input_data):
        self.eval()
        with torch.no_grad():
            outputs = self(input_data)
            _, predicted = torch.max(outputs.data, 1)
        return predicted

In [179]:
def train_test_split(data: pd.DataFrame, target_label : str, test_size=0.2, return_torch=None):
        
    # split the data into train and test
    train = data.sample(frac=(1-test_size),random_state=123)
    test = data.drop(train.index)
    
    # split the train and test into X and Y
    train_X = train.drop([target_label], axis=1).values
    train_Y = train[target_label].values
    test_X = test.drop([target_label], axis=1).values
    test_Y = test[target_label].values
        
    if return_torch:
        train_X = torch.tensor(train_X)
        train_Y = torch.tensor(train_Y)
        test_X = torch.tensor(test_X)
        test_Y = torch.tensor(test_Y)
        
    return train_X, train_Y, test_X, test_Y

# Load the data

In [180]:
# Congression Voting Dataset
cong_voting = pd.read_csv('./preprocessed-datasets/CongressionVoting_prepro.csv')

label2num = {"democrat": 0, "republican": 1}
num2label = {0: "democrat", 1: "republican"}

# convert the target label to numeric
cong_voting["class"] = cong_voting["class"].apply(lambda x: label2num[x])

In [181]:
cong_voting["class"].value_counts()

0    137
1     80
Name: class, dtype: int64

In [182]:
# Wine Quality Dataset
wine_quality = pd.read_csv('./preprocessed-datasets/wine_quality_prepro.csv', index_col=0)
wine_quality.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,class,wine_type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


## Run the model

In [213]:
# run the model for cong_voting dataset
train_X, train_Y, test_X, test_Y = train_test_split(cong_voting, "class", return_torch=True)

nodes_per_layer = [15, 6, 12]
input_shape = train_X.shape[1]
num_classes = 2 # binary classification
num_samples = train_X.shape[0] 

dataset = TensorDataset(train_X, train_Y) # wrap X and Y into a single dataset
train_loader = DataLoader(dataset, batch_size=32, shuffle=True) 
# in batch we have 32 samples, and we shuffle the data after the iteration over each batch
model = ClassificationModel(nodes_per_layer, input_shape, num_classes)
model.train_model(train_loader, num_epochs= 10, lr= 0.0001, val_loader=train_loader)
Y_pred = model.predict(test_X)
print(f"The f1 score is {metrics.f1_score(test_Y, Y_pred)}")

NameError: name 'init_weights' is not defined

In [206]:
# run the model for wine_quality dataset
train_X, train_Y, test_X, test_Y = train_test_split(wine_quality, "wine_type", return_torch=True)

nodes_per_layer = [5, 7, 5]
input_shape = train_X.shape[1]
num_classes = 10 # binary classification
num_samples = train_X.shape[0] 

dataset = TensorDataset(train_X, train_Y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ClassificationModel(nodes_per_layer, input_shape, num_classes)
model.train_model(train_loader, lr= 0.01, val_loader=train_loader)
Y_pred = model.predict(test_X)
print(f"The f1 score is {metrics.f1_score(test_Y, Y_pred)}")

TypeError: ClassificationModel.init_weights() takes 1 positional argument but 2 were given

In [67]:
model = ClassificationModel(nodes_per_layer, input_shape, num_classes)
print(model)

ClassificationModel(
  (model): Sequential(
    (0): Linear(in_features=12, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=7, bias=True)
    (3): ReLU()
    (4): Linear(in_features=7, out_features=5, bias=True)
    (5): ReLU()
    (6): Linear(in_features=5, out_features=2, bias=True)
    (7): Softmax(dim=1)
  )
)
